In [19]:
import pandas as pd
import numpy as np
import xlrd
import ipywidgets as widgets
import os

### Type filepath for target excel sheet ##

In [24]:
global tgtfile
tgtfile = "D:\\Sanna Lensu_2020\\sanna2020_brain\\results\\testi.xlsx"

### Type filepath for annotation results ##

In [21]:
global resultfile
resultfile = "D:\\Sanna Lensu_2020\\sanna2020_brain\\MAT\\Structure result-2092.txt"

In [25]:
global sheet
wb = xlrd.open_workbook(tgtfile)
sheet = wb.sheet_by_index(0)

In [40]:
def getCorners(sheet):
    corner = [0,0]
    for i in range(0, sheet.nrows):
        corner[0] = i;
        if (sheet.cell_value(i,0) != ""):
            break
    for i in range(0, sheet.ncols):
        corner[1] = i;
        if (sheet.cell_value(0,i) != ""):
            break
    return corner

In [41]:
getCorners(sheet)

[6, 65]

In [63]:
def getExcel ():
    try:
        #import_file_path = os.path.join(workingfolder, filename)
        df = pd.read_excel (tgtfile, header = getCorners(sheet)[0])
        df.rename(columns=lambda x: x.replace(".", "_"), inplace = True)
    except:
        raise
    return df

In [64]:
global df_target
df_target = getExcel()

In [67]:
df_target.shape

(93, 78)